<a href="https://colab.research.google.com/github/ekaratnida/DADS5001-Data-Analytics-and-Data-Science-Tools-and-Programming/blob/main/01_Data_Sources/ReadBTCdata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import time, requests, pandas as pd

INFO_URL = "https://api.hyperliquid.xyz/info"

In [6]:
def fetch_hl_candles(coin="BTC", interval="15m", days=7):
    now_ms = int(time.time() * 1000)
    print(f"now_ms = {now_ms}")
    start_ms = now_ms - days * 24 * 60 * 60 * 1000
    print(f"start_ms = {start_ms}")
    payload = {
        "type": "candleSnapshot",
        "req": {"coin": coin, "interval": interval, "startTime": start_ms, "endTime": now_ms},
    }

    print(payload)

    r = requests.post(INFO_URL, json=payload, timeout=15)
    r.raise_for_status()
    data = r.json()  # list of candles
    #print(data)
    df = pd.DataFrame(data)

    # rename and clean
    rename = {"t":"open_time","T":"close_time","o":"open","h":"high","l":"low","c":"close","v":"volume","n":"trades","i":"interval","s":"symbol"}
    df = df.rename(columns=rename)
    num_cols = ["open","high","low","close","volume"]
    for c in num_cols:
        df[c] = pd.to_numeric(df[c], errors="coerce")
    df["open_time"]  = pd.to_datetime(df["open_time"],  unit="ms", utc=True)
    df["close_time"] = pd.to_datetime(df["close_time"], unit="ms", utc=True)
    return df.sort_values("open_time")

In [7]:
df = fetch_hl_candles("BTC", "1h", days=7)
print(df.shape)
print(df.tail()) #[["open_time","open","high","low","close"]])


now_ms = 1769329156274
start_ms = 1768724356274
{'type': 'candleSnapshot', 'req': {'coin': 'BTC', 'interval': '1h', 'startTime': 1768724356274, 'endTime': 1769329156274}}
(169, 10)
                    open_time                       close_time symbol  \
164 2026-01-25 04:00:00+00:00 2026-01-25 04:59:59.999000+00:00    BTC   
165 2026-01-25 05:00:00+00:00 2026-01-25 05:59:59.999000+00:00    BTC   
166 2026-01-25 06:00:00+00:00 2026-01-25 06:59:59.999000+00:00    BTC   
167 2026-01-25 07:00:00+00:00 2026-01-25 07:59:59.999000+00:00    BTC   
168 2026-01-25 08:00:00+00:00 2026-01-25 08:59:59.999000+00:00    BTC   

    interval     open    close     high      low     volume  trades  
164       1h  88914.0  88974.0  89045.0  88900.0  113.49855    3733  
165       1h  88974.0  88958.0  89010.0  88824.0  186.43192    3885  
166       1h  88958.0  88803.0  88958.0  88609.0  563.86637    9348  
167       1h  88802.0  88615.0  88802.0  88600.0  301.95595    4344  
168       1h  88616.0  88658.0